## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [ ]:

import sqlite3
import pandas as pd

Loading the dataset for Box office mojo and Rotten tomatoes

In [5]:
#load dataset for Box Office Mojo
BoxOfficeMojo=pd.read_csv('zippedData/bom.movie_gross.csv.gz')
print(BoxOfficeMojo.head())

                                         title studio  domestic_gross  \
0                                  Toy Story 3     BV     415000000.0   
1                   Alice in Wonderland (2010)     BV     334200000.0   
2  Harry Potter and the Deathly Hallows Part 1     WB     296000000.0   
3                                    Inception     WB     292600000.0   
4                          Shrek Forever After   P/DW     238700000.0   

  foreign_gross  year  
0     652000000  2010  
1     691300000  2010  
2     664300000  2010  
3     535700000  2010  
4     513900000  2010  


In [6]:
#load dataset for Rotten Tomatoes
RottenTomatoes=pd.read_csv('zippedData/rt.reviews.tsv.gz',sep='\t',compression='gzip',encoding='latin-1')
print(RottenTomatoes.head())


   id                                             review rating   fresh  \
0   3  A distinctly gallows take on contemporary fina...    3/5   fresh   
1   3  It's an allegory in search of a meaning that n...    NaN  rotten   
2   3  ... life lived in a bubble in financial dealin...    NaN   fresh   
3   3  Continuing along a line introduced in last yea...    NaN   fresh   
4   3             ... a perverse twist on neorealism...     NaN   fresh   

           critic  top_critic         publisher               date  
0      PJ Nabarro           0   Patrick Nabarro  November 10, 2018  
1  Annalee Newitz           0           io9.com       May 23, 2018  
2    Sean Axmaker           0  Stream on Demand    January 4, 2018  
3   Daniel Kasman           0              MUBI  November 16, 2017  
4             NaN           0      Cinema Scope   October 12, 2017  


Inspecting the columns

In [7]:
print(BoxOfficeMojo.columns)


Index(['title', 'studio', 'domestic_gross', 'foreign_gross', 'year'], dtype='object')


In [8]:
print(RottenTomatoes.columns)

Index(['id', 'review', 'rating', 'fresh', 'critic', 'top_critic', 'publisher',
       'date'],
      dtype='object')


In [9]:
#check the info of the datasets
print(BoxOfficeMojo.info())
print(RottenTomatoes.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dt

#BoxOfficemojo dataset has 4 columns and RottenTomatoes dataset has 6 columns. Both datasets have no missing values.

Check for missing values

In [10]:
#Box office Mojo
print(BoxOfficeMojo.isnull().sum())
#Rotten Tomatoes
print(RottenTomatoes.isnull().sum())

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64
id                0
review         5563
rating        13517
fresh             0
critic         2722
top_critic        0
publisher       309
date              0
dtype: int64


Handle Missing Values

In [ ]:
#Remove rows missing important revenue data in Box Office Mojo dataset
BoxOfficeMojo_cleaned=BoxOfficeMojo.dropna(subset=['domestic_gross'])

In [17]:
#Fills missing values in Rotten Tomatoes dataset with 'Unknown'
RottenTomatoes_filled=RottenTomatoes.fillna('Unknown')


In [18]:
#drop the rows with missing values in Box Office Mojo dataset
BoxOfficeMojo_cleaned = BoxOfficeMojo.dropna()

In [19]:
#drop the rows with missing values in Rotten Tomatoes dataset
RottenTomatoes_cleaned = RottenTomatoes.dropna()

In [21]:
#fill missing scores in Rotten Tomatoes dataset with the mean score
RottenTomatoes_filled_scores = RottenTomatoes.copy()
RottenTomatoes_filled_scores['critic_score'] = RottenTomatoes_filled_scores['critic_score'].fillna(RottenTomatoes_filled_scores['critic_score'].mean())

KeyError: 'critic_score'

### BUSINESS UNDERSTANDING

##### KEY QUESTIONS FOR FOR MOVIE STUDIO STRATEGY
1. Which movie genres generate the highest revenue- by analyzing the total earnings fromm the different genres, we can determine which types of films are more profitable and likely to attact large audiences.
2. Does movie rating or critic score affect the performance- using ratings from Rotten tomatoes and movies data base , we can measure whether higher rated movies earn more revenue 
3. Does a longer runtime have an effect on rating or revenue- by analyzing the runtime in the dataset, we are able to identify which runtime generate higher earnings for different types of movies.
4. Does the release timing affect the earnings- by examining which months and their associated revene, we can identify the optimal times to launch fils for maximum success